최종본: 간력하게 함수로 보기 좋게 만듬

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re

#윈도우에서 열때
options = Options()
options.add_experimental_option('detach', True)  # 브라우저 바로 닫힘 방지
options.add_experimental_option('excludeSwitches', ['enable-logging'])  # 불필요한 메시지 제거

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

driver.get('https://www.imdb.com/title/tt0898266/episodes/?ref_=tt_eps_sm')


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from tqdm import tqdm 
import pandas as pd
import re

#리눅스 환경

url = "https://www.imdb.com/title/tt0898266/episodes/?ref_=tt_eps_sm"
driver = webdriver.Chrome(service=Service("/home/addinedu/dev_ws/driver/chromedriver-linux64/chromedriver"))

 

   

driver. get(url)

In [ ]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from tqdm import tqdm 

req = driver.page_source
soup = BeautifulSoup(req,'html.parser')  
all_box = soup.find(class_= "ipc-page-section ipc-page-section--base ipc-page-section--sp-pageMargin")
page_move = all_box.find(class_= 'ipc-tabs ipc-tabs--base ipc-tabs--align-left ipc-tabs--display-chip ipc-tabs--inherit')
move_on = page_move.find_all(class_= 'ipc-tab ipc-tab-link ipc-tab--on-base')

In [ ]:
#첫페이지로 이동
page_2dong = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[1]/div[2]/ul/a[1]')
page_2dong.click()

In [ ]:
def extract_episode_info(episode):
    title = episode.find('div', class_="ipc-title__text").get_text()
    day = episode.find('span', class_="sc-f2169d65-10 iZXnmI").get_text()
    rating = episode.find('span', class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating").get_text()

    rating_value = float(re.search(r'(\d+\.\d)', rating).group(1))

    return title, day, rating_value

# def navigate_to_page(driver, page_number):
#     page_2dong = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[1]/div[2]/ul/a[{page_number}]')
#     page_2dong.click()
    
def scrape_page(driver, page_number):
    try:
        try:
            page_2dong = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[1]/div[2]/ul/a[{page_number}]')
            page_2dong.click()
            time.sleep(3)
        except Exception as e:
            print(f"첫페이지 입니다 {page_number}: {e}")
        
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')  
        all_box = soup.find(class_="ipc-page-section ipc-page-section--base ipc-page-section--sp-pageMargin")
        all_inf = all_box.find(class_='sc-7b9ed960-0 jNjsLo')
        all_inf_2 = all_inf.find_all(class_='sc-282bae8e-1 dSEzwa episode-item-wrapper')

        title_list = []
        day_list = []
        rating_list = []

        for episode in all_inf_2:
            title, day, rating_value = extract_episode_info(episode)
            title_list.append(title)
            day_list.append(day)
            rating_list.append(rating_value)

        combined_list = list(zip(title_list, day_list, rating_list))
        return combined_list

    except Exception as e:
        print(f"Error while scraping page {page_number}: {e}")
        return None

In [ ]:


if __name__ == "__main__":
    a = len(move_on)  # Set the number of pages to move on
    all_data = []

    for i in range(a):
        result = scrape_page(driver, i+1)
        if result:
            all_data.extend(result)
            
    # navigate_to_page(driver, i+1)
    
    # Convert the list of tuples to a DataFrame
    df = pd.DataFrame(all_data, columns=['Title', 'Day', 'Rating'])

    # Print or use the DataFrame as needed
    print(df)

In [ ]:
driver.close()

In [ ]:
df.to_csv('C:/gg/movie2.csv',sep=',',encoding='utf-8')

In [ ]:
import pandas as pd
df = pd.read_csv('C:\gg\movie2.csv')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

df_sorted = df.sort_values(by='Rating', ascending=False)

# Select the top 10 titles
df_top10 = df_sorted.head(10)
# df_bottom10 = df_sorted.tail(10)
# Create a horizontal bar chart using Plotly Express
fig = px.bar(df_top10, x='Rating', y='Title', orientation='h',
             labels={'Rating': 'IMDb Rating', 'Title': 'Movie Title'},
             title='Top 10 IMDb Ratings by Title',
             height=600, width=800)


# Show the plot
fig.show()


In [ ]:
df_sorted = df.sort_values(by='Rating', ascending=False)

# Select the bottom 10 titles (lowest-rated) in ascending order
df_top10 = df_sorted.head(10)

# Create a box plot using Plotly Express for the bottom 10 titles
fig = px.box(df_top10, y='Rating', title='IMDb Ratings Distribution (Head 10 Titles)',
             labels={'Rating': 'IMDb Rating'})

# Show the plot
fig.show()

In [ ]:
fig = px.line(df, x='Day', y='Rating', title='IMDb Ratings Over Time',
              labels={'Rating': 'IMDb Rating', 'Day': 'Day'})


fig.show()

### 각 시즌별 회차별 평점

In [ ]:
for i in range(a):
    result = scrape_page(driver, i+1)
    if result:
        df = pd.DataFrame(result, columns=['Title', 'Day', 'Rating'])

        
        fig = px.bar(df, x='Title', y='Rating', title=f'IMDb Ratings - Page {i+1}',
                     labels={'Rating': 'IMDb Rating'})
        fig.update_layout(xaxis=dict(tickangle=45, tickmode='array', tickvals=list(df['Title'])))

        # Save the plot as an HTML file (optional)
        fig.write_html(f'imdb_ratings_page_{i+1}.html')

        # Show the plot
        fig.show()
        sns.swarmplot(x="Day", y="Rating", data=df, color=".5")